In [2]:
!pip install python-telegram-bot

  Using cached https://files.pythonhosted.org/packages/26/aa/82dfedea526128a55429527ca5e43c023846e00cc3df30c8dbfebe94bbb2/python_telegram_bot-12.2.0-py2.py3-none-any.whl


In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# This program is dedicated to the public domain under the CC0 license.

"""
First, a few callback functions are defined. Then, those functions are passed to
the Dispatcher and registered at their respective places.
Then, the bot is started and runs until we press Ctrl-C on the command line.
Usage:
Example of a bot-user conversation using ConversationHandler.
Send /start to initiate the conversation.
Press Ctrl-C on the command line or send a signal to the process to stop the
bot.
"""

import logging

from telegram import ReplyKeyboardMarkup
from telegram.ext import (Updater, CommandHandler, MessageHandler, Filters,
                          ConversationHandler)

# Enable logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    level=logging.INFO)

logger = logging.getLogger(__name__)

CHOOSING, TYPING_REPLY, TYPING_CHOICE = range(3)

reply_keyboard = [['Age', 'Favourite colour'],
                  ['Number of siblings', 'Something else...'],
                  ['Done']]
markup = ReplyKeyboardMarkup(reply_keyboard, one_time_keyboard=True)


reply_keyboard_2 = [""]
def facts_to_str(user_data):
    facts = list()

    for key, value in user_data.items():
        facts.append('{} - {}'.format(key, value))

    return "\n".join(facts).join(['\n', '\n'])


def start(update, context):
    update.message.reply_text(
        "Hi! My name is Doctor Botter. I will hold a more complex conversation with you. "
        "Why don't you tell me something about yourself?",
        reply_markup=markup)

    return CHOOSING


def regular_choice(update, context):
    text = update.message.text
    context.user_data['choice'] = text
    update.message.reply_text(
        'Your {}? Yes, I would love to hear about that!'.format(text.lower()))

    return TYPING_REPLY


def custom_choice(update, context):
    update.message.reply_text('Alright, please send me the category first, '
                              'for example "Most impressive skill"')

    return TYPING_CHOICE


def received_information(update, context):
    user_data = context.user_data
    text = update.message.text
    category = user_data['choice']
    user_data[category] = text
    del user_data['choice']

    update.message.reply_text("Neat! Just so you know, this is what you already told me:"
                              "{} You can tell me more, or change your opinion"
                              " on something.".format(facts_to_str(user_data)),
                              reply_markup=markup)

    return CHOOSING


def done(update, context):
    user_data = context.user_data
    if 'choice' in user_data:
        del user_data['choice']

    update.message.reply_text("I learned these facts about you:"
                              "{}"
                              "Until next time!".format(facts_to_str(user_data)))

    user_data.clear()
    return ConversationHandler.END


def error(update, context):
    """Log Errors caused by Updates."""
    logger.warning('Update "%s" caused error "%s"', update, context.error)


def main():
    # Create the Updater and pass it your bot's token.
    # Make sure to set use_context=True to use the new context based callbacks
    # Post version 12 this will no longer be necessary
    updater = Updater("984001690:AAHvJPz7H064B0J9CTJE-243qYMSOVhisPs", use_context=True)

    # Get the dispatcher to register handlers
    dp = updater.dispatcher

    # Add conversation handler with the states CHOOSING, TYPING_CHOICE and TYPING_REPLY
    conv_handler = ConversationHandler(
        entry_points=[CommandHandler('start', start)],

        states={
            CHOOSING: [MessageHandler(Filters.regex('^(Age|Favourite colour|Number of siblings)$'),
                                      regular_choice),
                       MessageHandler(Filters.regex('^Something else...$'),
                                      custom_choice)
                       ],

            TYPING_CHOICE: [MessageHandler(Filters.text,
                                           regular_choice)
                            ],

            TYPING_REPLY: [MessageHandler(Filters.text,
                                          received_information),
                           ],
        },

        fallbacks=[MessageHandler(Filters.regex('^Done$'), done)]
    )

    dp.add_handler(conv_handler)

    # log all errors
    dp.add_error_handler(error)

    # Start the Bot
    updater.start_polling()

    # Run the bot until you press Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT. This should be used most of the time, since
    # start_polling() is non-blocking and will stop the bot gracefully.
    updater.idle()


if __name__ == '__main__':
    main()

In [ ]:
import logging

import re

from telegram import ReplyKeyboardMarkup
from telegram.ext import (Updater, CommandHandler, MessageHandler, Filters,
                          ConversationHandler)

# Enable logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    level=logging.INFO)

logger = logging.getLogger(__name__)

CHOOSING, STEP_2_FIND_SPEC , CHOOSE_AGE, PROBLEM_CATEGORY, QUESTION_CATEGORY = range(5)
FIND_SPEC_NOW, CHOOSING_ABOUT = range(5,7)



def start(update, context):
    reply_keyboard = [['Найти специалиста', 'О сервисе']]

    markup = ReplyKeyboardMarkup(reply_keyboard, one_time_keyboard=True)
    update.message.reply_text(
        """Привет! Этот бот поможет тебе справиться с депрессией или другими психологическими проблемами, которые тебя беспокоят! 
        Давай начнем: выбери "Найти специалиста" или узнай о нас больше выбрав "О сервисе", 
        так же ты можешь выбрать быструю помощь и мы найдем тебе ближайшего освободившегося специалиста""",
        reply_markup=markup)

    return CHOOSING

def stop(update, context):
    """Закончить общение с ботом по команде /stop"""
    update.message.reply_text("""Пока!""")
    
    return ConversationHandler.END

def done(update, context):
    update.message.reply_text("""Пока!""")
    user_data = context.user_data
    if 'choice' in user_data:
        del user_data['choice']

    print(user_data)

    user_data.clear()
    return ConversationHandler.END

def Find_Spec(update, context):
    r"""найи специалиста выбор из двух альтернатив и переход к след шагу"""
    reply_keyboard_loc = [['Быстрая помощь','Подобрать', 'Список Специалистов']]

    markup_loc = ReplyKeyboardMarkup(reply_keyboard_loc, one_time_keyboard=True)
    
    update.message.reply_text(
        """Мы можем помочь подобрать Вам специалиста сразу или в зависимости от характера вашей проблемы, 
        либо вы можете выбрать его самостоятельно из списка специалистов.""",
        reply_markup=markup_loc)
    
    return STEP_2_FIND_SPEC

def Pol(update, context):
    r"""Пол"""
    reply_keyboard_loc = [['М', 'Ж']]
    markup_loc = ReplyKeyboardMarkup(reply_keyboard_loc, one_time_keyboard=True)
    
    update.message.reply_text("""Выберите ваш пол.""", reply_markup=markup_loc)
    return CHOOSE_AGE


def Age(update, context):
    r"""Возраст"""
    reply_keyboard_loc = [['< 16', '16-22', '22-40', "> 40"]]
    markup_loc = ReplyKeyboardMarkup(reply_keyboard_loc, one_time_keyboard=True)
    update.message.reply_text("""Выберите ваш возраст.""", reply_markup=markup_loc)
    
    return PROBLEM_CATEGORY

def Problem_Category(update, context):
    r"""Выбрать проблему из предложенных типов"""
    reply_keyboard_loc = [['Семья', 'Друзья', 'Отношения', "Работа", "Учеба", "Другое"]]
    markup_loc = ReplyKeyboardMarkup(reply_keyboard_loc, one_time_keyboard=True)
    update.message.reply_text("""К какой категории Вы могли бы отнести свою психологическую проблему?""", reply_markup=markup_loc)
    
    return QUESTION_CATEGORY

def Question_Category(update, context):
    r"""Выбрать вопрос из предложенных типов"""
    reply_keyboard_loc = [['Я запутался/лась', 'Мне не интересно или скучно', 'Я устал', "Меня никто не слушает",
                           "Что делать дальше", "Как сделать правильный выбор", "Другое"]]
    markup_loc = ReplyKeyboardMarkup(reply_keyboard_loc, one_time_keyboard=True)
    update.message.reply_text("""Какие утверждения или вопросы подходят Вам наиболее всего?""", reply_markup=markup_loc)
    
    return WAITING_TIME

def find_spec_now(update, contex):
    r"""Подбор специалиста сразу"""
    update.message.reply_text("""Мы уже ищем специалиста. Ожидайте...""")
    ''' Нужно допилить алгоритм быстрого поиска'''
    def fast_search(update, context):
        pass
    
    return FIND_SPEC_NOW

def about(update, context):
    r"""О нас"""
    reply_keyboard_loc = [['О сервисе', 'Стоимость услуг', 'Отзывы', "О нас пишут", "FAQ" ]]
    markup_loc = ReplyKeyboardMarkup(reply_keyboard_loc, one_time_keyboard=True)
    update.message.reply_text("""PsyHelp - бот психологической поддержки""", reply_markup=markup_loc)
    
    return CHOOSING_ABOUT
    
def error(update, context):
    """Log Errors caused by Updates."""
    logger.warning('Update "%s" caused error "%s"', update, context.error)


def main():
    # Create the Updater and pass it your bot's token.
    # Make sure to set use_context=True to use the new context based callbacks
    # Post version 12 this will no longer be necessary
    updater = Updater("984001690:AAHvJPz7H064B0J9CTJE-243qYMSOVhisPs", use_context=True)

    # Get the dispatcher to register handlers
    dp = updater.dispatcher

    # Add conversation handler with the states CHOOSING, TYPING_CHOICE and TYPING_REPLY
    conv_handler = ConversationHandler(
        entry_points=[CommandHandler('start', start)],

        states={
            CHOOSING: [MessageHandler(Filters.regex('^Найти специалиста$'),
                                      Find_Spec),
                       MessageHandler(Filters.regex('^О сервисе$'),
                                      about),
                       ],
            STEP_2_FIND_SPEC: [ MessageHandler(Filters.regex('^Быстрая помощь$'),
                                      find_spec_now),
                               MessageHandler(Filters.regex('^Подобрать$'),
                                      Pol),
                       MessageHandler(Filters.regex('^Список Специалистов$'),
                                      Pol)
            ],

            CHOOSE_AGE: [MessageHandler(Filters.regex('^.*$'), Age) 
                            ],
            
            PROBLEM_CATEGORY: [MessageHandler(Filters.regex('^.*$'), Problem_Category)
                            ],
            
            QUESTION_CATEGORY: [MessageHandler(Filters.regex('^.*$'), Question_Category)
                            ],
        },

        fallbacks=[MessageHandler(Filters.regex('^Done$'), done),
                  CommandHandler('stop', stop)]
    )

    dp.add_handler(conv_handler)

    # log all errors
    dp.add_error_handler(error)

    # Start the Bot
    updater.start_polling()

    # Run the bot until you press Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT. This should be used most of the time, since
    # start_polling() is non-blocking and will stop the bot gracefully.
    updater.idle()


if __name__ == '__main__':
    main()

2019-12-05 22:52:32,858 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - WARNING - Retrying (Retry(total=2, connect=None, read=None, redirect=None)) after connection broken by 'ConnectTimeoutError(<telegram.vendor.ptb_urllib3.urllib3.connection.VerifiedHTTPSConnection object at 0x0000016555C50A48>, 'Connection to api.telegram.org timed out. (connect timeout=5.0)')': /bot984001690:AAHvJPz7H064B0J9CTJE-243qYMSOVhisPs/getUpdates
2019-12-05 22:52:37,876 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - WARNING - Retrying (Retry(total=1, connect=None, read=None, redirect=None)) after connection broken by 'ConnectTimeoutError(<telegram.vendor.ptb_urllib3.urllib3.connection.VerifiedHTTPSConnection object at 0x0000016555AEEF88>, 'Connection to api.telegram.org timed out. (connect timeout=5.0)')': /bot984001690:AAHvJPz7H064B0J9CTJE-243qYMSOVhisPs/getUpdates
2019-12-05 22:52:42,877 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - WARNING - Retrying (Retry(total=0, connect=None, 

2019-12-08 09:06:22,604 - __main__ - WARNING - Update "None" caused error "urllib3 HTTPError HTTPSConnectionPool(host='api.telegram.org', port=443): Max retries exceeded with url: /bot984001690:AAHvJPz7H064B0J9CTJE-243qYMSOVhisPs/getUpdates (Caused by ConnectTimeoutError(<telegram.vendor.ptb_urllib3.urllib3.connection.VerifiedHTTPSConnection object at 0x0000016555AEEB48>, 'Connection to api.telegram.org timed out. (connect timeout=5.0)'))"
2019-12-08 09:06:29,111 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - WARNING - Retrying (Retry(total=2, connect=None, read=None, redirect=None)) after connection broken by 'ConnectTimeoutError(<telegram.vendor.ptb_urllib3.urllib3.connection.VerifiedHTTPSConnection object at 0x0000016555D98CC8>, 'Connection to api.telegram.org timed out. (connect timeout=5.0)')': /bot984001690:AAHvJPz7H064B0J9CTJE-243qYMSOVhisPs/getUpdates
2019-12-08 09:06:34,123 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - WARNING - Retrying (Retry(total=1, connec

2019-12-08 09:07:54,937 - telegram.ext.updater - ERROR - Error while getting Updates: urllib3 HTTPError HTTPSConnectionPool(host='api.telegram.org', port=443): Max retries exceeded with url: /bot984001690:AAHvJPz7H064B0J9CTJE-243qYMSOVhisPs/getUpdates (Caused by ConnectTimeoutError(<telegram.vendor.ptb_urllib3.urllib3.connection.VerifiedHTTPSConnection object at 0x0000016555AA13C8>, 'Connection to api.telegram.org timed out. (connect timeout=5.0)'))
2019-12-08 09:07:54,937 - __main__ - WARNING - Update "None" caused error "urllib3 HTTPError HTTPSConnectionPool(host='api.telegram.org', port=443): Max retries exceeded with url: /bot984001690:AAHvJPz7H064B0J9CTJE-243qYMSOVhisPs/getUpdates (Caused by ConnectTimeoutError(<telegram.vendor.ptb_urllib3.urllib3.connection.VerifiedHTTPSConnection object at 0x0000016555AA13C8>, 'Connection to api.telegram.org timed out. (connect timeout=5.0)'))"
2019-12-08 09:08:07,546 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - WARNING - Retrying (Ret

2019-12-08 09:10:11,771 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - WARNING - Retrying (Retry(total=0, connect=None, read=None, redirect=None)) after connection broken by 'ConnectTimeoutError(<telegram.vendor.ptb_urllib3.urllib3.connection.VerifiedHTTPSConnection object at 0x000001655620F4C8>, 'Connection to api.telegram.org timed out. (connect timeout=5.0)')': /bot984001690:AAHvJPz7H064B0J9CTJE-243qYMSOVhisPs/getUpdates
2019-12-08 09:10:16,783 - telegram.ext.updater - ERROR - Error while getting Updates: urllib3 HTTPError HTTPSConnectionPool(host='api.telegram.org', port=443): Max retries exceeded with url: /bot984001690:AAHvJPz7H064B0J9CTJE-243qYMSOVhisPs/getUpdates (Caused by ConnectTimeoutError(<telegram.vendor.ptb_urllib3.urllib3.connection.VerifiedHTTPSConnection object at 0x0000016555AA1248>, 'Connection to api.telegram.org timed out. (connect timeout=5.0)'))
2019-12-08 09:10:16,783 - __main__ - WARNING - Update "None" caused error "urllib3 HTTPError HTTPSConnectionPo

2019-12-08 09:13:35,399 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - WARNING - Retrying (Retry(total=1, connect=None, read=None, redirect=None)) after connection broken by 'ConnectTimeoutError(<telegram.vendor.ptb_urllib3.urllib3.connection.VerifiedHTTPSConnection object at 0x0000016555AEEEC8>, 'Connection to api.telegram.org timed out. (connect timeout=5.0)')': /bot984001690:AAHvJPz7H064B0J9CTJE-243qYMSOVhisPs/getUpdates
2019-12-08 09:13:40,411 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - WARNING - Retrying (Retry(total=0, connect=None, read=None, redirect=None)) after connection broken by 'ConnectTimeoutError(<telegram.vendor.ptb_urllib3.urllib3.connection.VerifiedHTTPSConnection object at 0x0000016555AEEB08>, 'Connection to api.telegram.org timed out. (connect timeout=5.0)')': /bot984001690:AAHvJPz7H064B0J9CTJE-243qYMSOVhisPs/getUpdates
2019-12-08 09:13:45,423 - telegram.ext.updater - ERROR - Error while getting Updates: urllib3 HTTPError HTTPSConnectionPool(host